# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

**You can use ChatGPT for this one**, but I'd first try to do it on your own. Link to any conversations you have with ChatGPT. **Use the share button in the top right – not just the URL of your chat**.

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`. You might find [this page helpful](https://jonathansoma.com/everything/scraping/pagination/), although the "On an interactive site" example uses Selenium instead of Playwright. You should be able to figure out how to change it!
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!
* **Tip:** Try to figure it out yourself before you submit to the gods of AI

In [26]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/tools_search/mccs_search.asp")
await page.fill("input[id='namedata']", "WRECK")
await page.select_option('select[name="name_carrier_type"]', {'label': 'DBA Name'})
#await page.click("button[name='searchtype']")
await page.click("button[type='submit']")
from bs4 import BeautifulSoup
html = await page.content()
from bs4 import BeautifulSoup
table = soup_doc.find("div",id_="content")
print(table)

TimeoutError: Page.click: Timeout 30000ms exceeded.
Call log:
waiting for locator("button[type='submit']")


### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

## Save as `wreckers.csv`

# Part Two: Company info

> You can use whatever tool you'd like for this, but form submission doesn't necessarily mean Playwright! If you want to go the `requests` route instead, feel free to [look at this page](https://jonathansoma.com/everything/scraping/pretending-to-be-a-browser/) about several ways to pretend to be a browser.

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. Also include `TDLR Number` in the dictionary. **Print the dictionary**.

> ***Tip:** It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on***
>
> ***Tip:** Be sure you don't forget to include `TDLR Number` in the dictionary!*

## Step 2: Move into one cell

Move the code above all into one cell that relies on the variable `tdlr_number`.

Add the code below to the page and confirm that it displays the data for the correct result.

```python
tdlr_number = '0654479VSF'
```

Confirm that the information is correct. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

## Scrape every single row, creating a list of dictionaries from the scraped data.

You should never use for loops with pandas *except when working with Playwright*. To use loops in pandas you'll make use of `.iterrows()`.

The code below loops through a dataframe called `df` and prints out the `address` column.

```python
for index, row in df.iterrows():
    print(row['address'])
```

You'll adapt this code to use your dataframe, and combine it with the scraping code you wrote above.

> ***Tip:** This is like what we did for the townships in class*

## Save your dataframe as `data-uncleaned.csv`

# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df = df.drop(columns=[...])`)

## Combine with the original CSV file

To combine dataframes based on indexes, you use `df.join(other_df)`. If you'd prefer to match by columns, you could also use `df.merge(other_df, left_on='...', right_on='...')` and tell it the two columns that match between the two.

### Save to a CSV